In [1]:
import pandas as pd
import Common_Functions as cmn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [112]:

df_change = pd.read_csv(cmn.PATH_df_no_NAN_99p_YoY).drop(columns="Unnamed: 0")
#df_change = df_change.drop('Volume YoY change',axis =1)
print(df_change.shape)
print(df_change.columns.tolist())

(41430, 30)
['ISIN', 'Year', 'Total Return', 'ESG Score', 'ESG Combined Score', 'ESG Controversies Score', 'Social Pillar Score', 'Governance Pillar Score', 'Environmental Pillar Score', 'Revenue - Mean', 'Earnings Per Share - Mean', 'Return On Equity - Mean', 'Volume', 'Company Market Cap', 'EBITDA - Mean', 'Country of Headquarters', 'Region of Headquarters', 'TRBC Industry Name', 'TRBC Industry Group Name', 'TRBC Business Sector Name', 'TRBC Economic Sector Name', 'ESG Score YoY change', 'ESG Combined Score YoY change', 'ESG Controversies Score YoY change', 'Social Pillar Score YoY change', 'Governance Pillar Score YoY change', 'Revenue - Mean YoY change', 'Earnings Per Share - Mean YoY change', 'Company Market Cap YoY change', 'EBITDA - Mean YoY change']


In [113]:
def prediction_int(df, feature):
    pred = ((df[feature] > 0).astype(int))
    return pred.astype(int)

In [114]:

feature_cols = df_change.filter(regex='Return|YoY').columns.tolist()
print(feature_cols)

['Total Return', 'Return On Equity - Mean', 'ESG Score YoY change', 'ESG Combined Score YoY change', 'ESG Controversies Score YoY change', 'Social Pillar Score YoY change', 'Governance Pillar Score YoY change', 'Revenue - Mean YoY change', 'Earnings Per Share - Mean YoY change', 'Company Market Cap YoY change', 'EBITDA - Mean YoY change']


In [115]:
df_change_with_classifier = df_change.copy()
for feature in feature_cols:
    df_change_with_classifier[f"{feature} Classified"] = prediction_int(df_change, feature)
    
df_change_with_classifier.shape

(41430, 41)

In [123]:
ESG_List = ['ESG Score',
            'ESG Combined Score',
            'ESG Controversies Score',
            'Social Pillar Score',
            'Governance Pillar Score',
            'ESG Score YoY change',
            'ESG Combined Score YoY change',
            'ESG Controversies Score YoY change',
            'Social Pillar Score YoY change',
            'Governance Pillar Score YoY change']

Financials_List = [ 'Total Return Classified', 
                    'Return On Equity - Mean Classified',                    
                    'Revenue - Mean YoY change Classified',
                    'Earnings Per Share - Mean YoY change Classified',
                    'Company Market Cap YoY change Classified',
                    'EBITDA - Mean YoY change Classified']

In [129]:
def CustomClassifier(X, y_list):
   result = {}
   for y in y_list:
      
      X_train, X_test, y_train, y_test = train_test_split(X, y_list[y], test_size = 0.20, random_state = 101)
      rfc = RandomForestClassifier(n_estimators=100, random_state=42)

      # fit random forest classifier to training data
      rfc.fit(X_train, y_train)

      # make predictions on test data
      y_pred = rfc.predict(X_test)

      # calculate accuracy of predictions
      result[y] = accuracy_score(y_test, y_pred)

   return result   


In [130]:
X = df_change_with_classifier[ESG_List]
y_list = df_change_with_classifier[Financials_List]

results = CustomClassifier(X, y_list)
print(results)

{'Total Return Classified': 0.5966690803765388, 'Return On Equity - Mean Classified': 0.929640357229061, 'Revenue - Mean YoY change Classified': 0.6932174752594739, 'Earnings Per Share - Mean YoY change Classified': 0.5696355298093169, 'Company Market Cap YoY change Classified': 0.5627564566739078, 'EBITDA - Mean YoY change Classified': 0.6466328747284577}
